Generate Cubes

In [ ]:
#Import libraries

import ee 
import geemap
import xee
import xarray as xr
import datetime
import rioxarray as rxr
import pandas as pd
import matplotlib.pyplot as plt
import os
import ipyleaflet


local_dir = "C:/Users/jaden/Downloads/Research/Notebooks/GEE/tifs"
os.makedirs(local_dir, exist_ok=True)

In [2]:
#Authenticate/initialize earth engine
ee.Authenticate()


True

In [4]:
ee.Initialize(
    project = 'sar-testing-472822',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
Map = geemap.Map(basemap = 'TERRAIN')
Map #to view map, selected point in US

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

Set bounding box in map widget above: for testing purposes, kennicott area
Note that the GRD image selection will include any scenes that *intersect* the box bounds

In [3]:
# Get the last drawn feature
loc = Map.draw_last_feature.geometry()

In [65]:
#Get GRD images
path = 14 #set path, same as ASF number
frame = 5 #set frame [WIP], requires trial and error to determine the correct correspondence
#https://forum.earthdata.nasa.gov/viewtopic.php?t=4161 for more info
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(loc)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('relativeOrbitNumber_start', path))
    .filter(ee.Filter.eq('sliceNumber', frame))
    .select('VH')
)

#reproject to lower resolution
def downsample(img):
    # Reduce resolution using mean reducer
    img_lowres = img.reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=10000,  # controls aggregation of high-res pixels
        
    )
    # Reproject to desired CRS and scale
    img_lowres = img_lowres.reproject(
        crs='EPSG:32607',
        scale=1000  # 500m pixels
    )
    return img_lowres

img_vv = img_vv.map(downsample)
# Extract distinct sliceNumbers for trial/error finding correct frame
slice_numbers = (img_vv
                 .aggregate_array("sliceNumber")
                 .distinct()
                 .sort())

data_window = ee.Filter.date('2019-11-01', '2020-01-01')
img_vv = img_vv.filter(data_window)

print("Distinct sliceNumbers:", slice_numbers.getInfo())
print("Number of images:", img_vv.size().getInfo())


frame = img_vv.first().reproject(crs="EPSG:32607", scale=10000)
m = geemap.Map()
#m.centerObject(avg_frame, 5)
m.add_layer(frame, {'min':-25, 'max':5}, 'SAR',True)
m

Distinct sliceNumbers: [5]
Number of images: 5


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [48]:
#Get bounds of union of image collection data
footprint_union = img_vv.geometry().dissolve()
#reset Map
m = geemap.Map()
# Add the union footprint
m.addLayer(footprint_union, {'color': 'red'}, 'S1 Footprint Union')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [66]:
task = ee.batch.Export.image.toDrive(
    image=frame,
    description='export_s1_low_res',
    region=frame.geometry(),
    scale=10000  # <- controls output resolution
)
task.start()

In [6]:
#Get DEM of GRD union bounds:

dem_clipped = (
    ee.ImageCollection("COPERNICUS/DEM/GLO30") 
    .filterBounds(footprint_union)
    .select('DEM')
)


single_dem = dem_clipped.mosaic().toFloat()

#CMU colors
vis_params = {
    'min': 0,
    'max': 3000,
    'palette': ['blue', 'green', 'white']
}

#Sanity check
centroid = footprint_union.centroid().coordinates().getInfo()
lon, lat = centroid[0], centroid[1]
m.setCenter(lon, lat, zoom=5)
m.addLayer(single_dem, vis_params, 'DEM')
m

Map(center=[61.609210398328756, -143.1184445951888], controls=(WidgetControl(options=['position', 'transparent…

In [59]:
#process glacier files

#declare glacier id func
def add_numeric_id(f):
    rgi_id = ee.String(f.get("rgi_id"))  # e.g. "RGI2000-v7.0-G-11-00351"
     # Split on "-"
    parts = rgi_id.split("-")   # returns ee.List

    # Get region and number explicitly by index
    region = ee.String(parts.get(3))   # "11"
    number = ee.String(parts.get(4))   # "00351"

    # Convert to integer
    glac_id = ee.Number.parse(number)

    return f.set("glac_id", glac_id)



rgi_vector = (
    ee.FeatureCollection("projects/sat-io/open-datasets/RGI/RGI_VECTOR_MERGED_V7")
    .filterBounds(footprint_union)
)

rgi_with_id = rgi_vector.map(add_numeric_id)

rgi = rgi_with_id.reduceToImage(
    properties=["glac_id"],
    reducer=ee.Reducer.first()
).unmask(0) 

rgi_geo = rgi_vector.geometry().dissolve().simplify(maxError=250)

#rgi = rgi_vector.reduceToImage(
#    properties= ['area_km2'],
#    reducer= ee.Reducer.first()
#)

vis_params = {
    'min': 0,
    'max': 100000,
    'palette': ['black', 'green', 'white']
}
#rgi = rgi.clip(footprint_union)
GlacierMap = geemap.Map()
rgi_binary = rgi.gt(0).unmask()
GlacierMap.addLayer(rgi, vis_params, 'Glacier raster mask', True)
#GlacierMap.addLayer(foot,vis_params,'Glacier Features')
GlacierMap #takes a while to load all of them

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [31]:
#this will take a long time!!
BoundsMap = geemap.Map()
rgi_styled = rgi_vector.style(
    color="red",       # outline color
    width=2,           # outline width
    fillColor="00000000"  # transparent fill
)
#BoundsMap.addLayer(
    #rgi_geo,
    #{'color':'red'},
    #'rgi_vector_outlines',
    #True
#)
#BoundsMap #


In [61]:
#Clip grd to glaciers
img_vv_clipped = img_vv.map(lambda img: img.clip(rgi_geo))


frame = img_vv_clipped.first()
task = ee.batch.Export.image.toDrive(
    image=frame,
    description='export_s1_2',
    region=frame.geometry(),
    scale=1000  # <- controls output resolution
)
task.start()

In [64]:
frame.pixelArea().get('area')

In [35]:
BoundsMap.centerObject(img_vv.first(), 5)
BoundsMap.add_layer(img_vv_clipped.first(), {'min':-25, 'max':5}, 'SAR',True)
BoundsMap #also takes a while, but this works

Map(bottom=18865.0, center=[61.60926934507247, -143.11894639699617], controls=(WidgetControl(options=['positio…

Generate video

In [13]:


video_args = {
  'dimensions': 768,
  'region': footprint_union,
  'framesPerSecond': 7,
  'crs': 'EPSG:3857',
  'min': -40.0,
  'max': 35.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

animation_url = img_vv.filterDate('2019-01-01', '2020-01-02').getVideoThumbURL(video_args)
print(f"Animation URL: {animation_url}")

Animation URL: https://earthengine-highvolume.googleapis.com/v1/projects/sar-testing-472822/videoThumbnails/fb17dd3d57f74470dc921eeb57414d36-22c3d6c096da56cb41ef29a88a3d2c29:getPixels


#Generate xarray

In [16]:
img_vv_clipped.first().projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32607',
 'transform': [10, 0, 239754.34207127654, 0, -10, 6944241.2863683915]}

In [19]:
ds = geemap.ee_to_xarray(
    img_vv_clipped,
    projection='EPSG:32607',
    scale=100,
    geometry=rgi_geo
)

In [20]:
ds

<xarray.Dataset> Size: 159MB
Dimensions:  (time: 5, lon: 3142, lat: 2534)
Coordinates:
  * time     (time) datetime64[ns] 40B 2019-11-09T15:56:27 ... 2019-12-27T15:...
  * lon      (lon) float64 25kB 2.476e+05 2.477e+05 ... 5.616e+05 5.617e+05
  * lat      (lat) float64 20kB 6.668e+06 6.668e+06 ... 6.921e+06 6.921e+06
Data variables:
    VH       (time, lon, lat) float32 159MB ...
Attributes:
    crs:      EPSG:4326

In [23]:
ds.to_netcdf("test.nc")

KeyboardInterrupt: 

Merge DEM and export data!

In [32]:
single_dem = dem_clipped.mosaic().toFloat()
def add_dem(img):
    return img.addBands(single_dem.rename('DEM'))
merged_coll = img_vv.map(add_dem)

def cast_bands(img):
    return img.toFloat()  # converts all bands in img to Float32
merged_coll = merged_coll.map(cast_bands)

print(type(merged_coll))
print("Number of images:", merged_coll.size().getInfo())




<class 'ee.imagecollection.ImageCollection'>
Number of images: 79


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [ ]:
glacier_ids = rgi.aggregate_array('glac_id')
print(glacier_ids.getInfo()) #might take a while

[0, 12965, 12088, 6016, 12954, 13838, 6046, 12905, 5589, 16317, 6338, 5740, 16307, 6010, 6127, 14010, 14216, 5599, 5749, 5955, 12610, 5825, 13666, 6120, 5979, 12420, 5959, 5492, 5849, 5536, 12791, 13659, 13000, 6058, 6340, 14180, 13094, 12207, 13916, 6051, 6014, 5702, 5833, 5843, 12254, 5906, 5780, 6135, 16515, 5875, 5694, 14136, 13930, 16437, 16312, 11857, 16634, 5964, 6125, 13089, 13110, 5761, 6114, 6047, 14101, 5897, 11974, 6165, 5720, 12684, 5685, 6255, 5572, 6044, 16554, 5669, 14183, 16441, 16646, 16323, 11952, 13130, 14203, 6052, 5966, 16822, 5703, 12273, 6064, 16413, 13956, 12311, 11761, 12890, 5878, 16346, 5525, 12163, 12841, 16486, 5584, 6349, 6308, 5535, 5673, 5724, 12089, 6160, 13944, 12972, 5641, 6001, 12899, 6344, 6352, 9730, 6333, 13826, 12322, 11696, 13673, 12618, 5999, 6121, 12964, 16537, 12136, 6009, 14080, 12984, 11796, 12416, 5839, 5514, 5981, 6298, 5663, 12277, 6146, 11707, 5558, 5490, 11937, 12615, 13954, 13983, 6031, 16406, 16420, 11822, 6181, 14218, 12821, 6107, 

In [49]:
ds_rgi = xr.open_dataset(
    rgi,
    engine='ee',
    crs='EPSG:4326',
    scale=30,
    geometry=footprint_union
)
ds_rgi


<xarray.Dataset> Size: 28B
Dimensions:  (time: 1, lon: 1, lat: 1)
Coordinates:
  * time     (time) int64 8B 0
  * lon      (lon) float64 8B -130.7
  * lat      (lat) float64 8B 75.64
Data variables:
    first    (time, lon, lat) float32 4B ...
Attributes:
    crs:      EPSG:4326

Merge Glaciers

In [ ]:


def add_rgi(img):
    return img.addBands(rgi.rename('rgi_ind_glacier_mask'))
full_merged_coll = merged_coll.map(add_rgi)


ds_merged = xr.open_dataset(
    full_merged_coll,
    engine='ee',
    crs='EPSG:4326',
    scale=30,
    geometry=footprint_union,
)
#ds

# Save to NetCDF
ds_merged.to_netcdf("testGEE_GRD_merged.nc")
